Experiment,Variable to Change,Fixed Variables,Purpose
1. Sensitivity,"Threshold % (0, 1, 5, 10, 20)",Model: 1.5B  Method: NF4,Find the optimal trade-off (Sweet Spot).
2. Methods,"Method (NF4, AWQ, GPTQ)",Model: 1.5B  Threshold: 5% (or Sweet Spot),Compare KLD impact on different quantizers.
3. Scaling,"Model Size (1.5B, 7B)",Method: (Best of Exp 2)  Threshold: 5%,Test if larger models behave differently.

# **Setup & Dependencies**

**We use L4 GPU**

In [1]:
!pip uninstall transformers torch torchaudio torchvision wandb -y
!pip install llmcompressor
!pip install -q accelerate bitsandbytes datasets scipy matplotlib wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset, concatenate_datasets
from datasets import Dataset
import copy
import gc
import time
from tqdm import tqdm
import shutil
import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: wandb 0.23.1
Uninstalling wandb-0.23.1:
  Successfully uninstalled wandb-0.23.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.2/273.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Set for reproducibility
import random
import numpy as np
from transformers import set_seed

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
set_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## **Configuration & Experiment Controls**

# Preliminary Check:
Justifying the "Base" Precision
Variable: Floating Point Type (FP8 vs. FP4/NF4)

Purpose: Before running complex KLD experiments, you must decide what your "base" low-precision format is. If FP4 destroys the model completely and FP8 is perfect, then KLD is needed for FP4 but not FP8. If both are good, FP4 is better for efficiency.

Design:

Run: 1.5B Model (FP16 Baseline) vs. 1.5B (FP8) vs. 1.5B (NF4).

Metric: Perplexity & MMLU Accuracy.

Hypothesis: NF4 offers higher compression but higher degradation than FP8. This justifies using NF4 (or INT4 methods) as the primary candidate for your KLD restoration because it needs the help more than FP8 does.

Decision: If verified, fix 4-bit (NF4/INT4) as the standard base for the rest of the experiments.

**Refer to the FP8 vs FP4 notebook**

**Results: NF4 offers higher compression but higher degradation than FP8. We will use NF4 (or INT4 methods) as the primary candidate for the following experiments because it needs the help more than FP8 does.**

# Experiment 1: Sensitivity Analysis
Research Question: How much of the model actually needs to be kept in high precision to recover performance? Is there a point of diminishing returns?

Fixed Variables:

Model: Qwen2.5-1.5B (Small enough to run fast, big enough to show signal).

Method: NF4 (The simplest 4-bit baseline).

Independent Variable (Change this):

KLD Threshold / % Restored: 0% (Baseline), 1%, 5%, 10%, 20%.


**Refer to the FP8 vs FP4 notebook**


# Experiment 2: Algorithm Comparison
Research Question: Does KLD guidance work better on top of simple quantization (NF4) or advanced quantization (AWQ/GPTQ)?

Fixed Variables:

Model: Qwen2.5-1.5B (Consistent with Exp 1).

Threshold: Fix this to the "winner" from Exp 1 (e.g., if 5% was the sweet spot, use 5% for all).

Independent Variable (Change this):

Method: NF4 vs. AWQ vs. GPTQ.

Why: AWQ and GPTQ already do some optimization. You want to see if your KLD method adds value on top of them, or if it's only useful for naive methods like NF4.

# Experiment 3: Model Size
Research Question: Does this technique generalize to larger models? (Larger models are usually more robust to quantization; do they need less restoration?)

Fixed Variables:

Method: The "Winner" from Exp 2 (likely NF4 for speed or AWQ for accuracy).

Threshold: Fix to the "sweet spot" (e.g., 5%).

Independent Variable (Change this):

Model Size: 1.5B vs. 7B.